In [1]:
# libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
    
import re

from tqdm import tqdm
import time

import sqlite3

import os
import random
import string

In [2]:
# PANDAS OPTIONS
# Set maximum number of columns and rows to display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Set the maximum column width to a high value
pd.set_option('display.max_colwidth', 1000)

### data import

In [29]:
# read back raw data from csv
df = pd.read_csv('scraped_data_clean.csv')

In [4]:
df.head(2)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price
0,BOITE DE 325 CARTOUCHES 22LR FEDERAL GOLD PLATED HOLLOW POINT 36 GRAINS,https://www.naturabuy.fr/BOITE-DE-325-CARTOUCHES-22LR-FEDERAL-GOLD-PLATED-HOLLOW-POINT-36-GRAINS-item-10619103.html,Federal,1,46.9,12.0,Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Creuse boite de 325 cartouches de marque Federal 22lr 36 grains round nose hollow point cuivree excellente cartouche pour le tir dans les armes a verrou comme semi automatique marque : Fédéral référence : 725 Type d’ogive round nose cuivree a pointe creuse vitesse 384 mètres par secondes poids ogive : 36 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation,58.9
1,Boîte ancienne 22 LR Remington 22 Target,https://www.naturabuy.fr/Boite-ancienne-22-LR-Remington-22-Target-item-10653966.html,Remington,0,11.0,4.4,Marque : RemingtonEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte en parfait état. Envoie par mondial relais 4€40,15.4


### Stop at 1st match
new way 1

In [6]:
# detects number divisible by 50 w\o remainder, and 525 and 325
# 0 value if nothing was found
# STOPS @ 1st

def find_ammo_quantity(text):
    pattern = r'\b(50|525|325)\b'  # Regular expression pattern to match required numbers
    match = re.search(pattern, text)
    return int(match.group()) if match else 0

def process_first_match(df):
    df['ammo_qtty_title'] = df['product_name'].apply(find_ammo_quantity)
    df['ammo_qtty_description'] = df['product_description'].apply(find_ammo_quantity)
    return df

df_v1 = process_first_match(df)

In [7]:
df_v1.sample(5)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title,ammo_qtty_description
1222,50 Munitions CCI Varmint Stinger 32G Cal 22 Lr HP,https://www.naturabuy.fr/50-Munitions-CCI-Varmint-Stinger-32G-Cal-22-Lr-HP-item-9338067.html,CCI,1,21.00,12.00,Marque : CCIEtat de l'objet : NeufType : StandardType d'ogive : Cuivre Boîte de 50 balles CCI Varmint Stinger Calibre : 22LR Poids : 2.1 grammes - 30 grains Type d'Ogive : Hollow Point Cuivrée Produit peut être vendu sur un autre support MERCI DE NOUS CONSULTER POUR VERIFIER L'ETAT DE STOCK,33.00,50,50
1116,Munitions Cal.22lr Eley high velocity hollow par 500,https://www.naturabuy.fr/Munitions-Cal-22lr-Eley-high-velocity-hollow-par-500-item-10735886.html,Eley,1,88.00,10.90,"Marque : EleyEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Plomb Calibre .22 LR Conditionnement Boîte en plastique de 50 cartouches Type Ogive HP, Plomb Poids du projectile 38 grains, soit 2.59 grammes Percussion Annulaire Amorçage Annulaire Type d'étuis Black oxidized 500 munitions soit 10 boites de 50",98.90,0,50
1179,50 cartouches CCI 22LR Short range green HP - 21 grains plinking,https://www.naturabuy.fr/50-cartouches-CCI-22LR-Short-range-green-HP-21-grains-plinking-item-10647763.html,CCI,1,13.09,7.49,"Marque : CCIEtat de l'objet : NeufType : PolymèreType d'ogive : Cuivre 50 cartouches CCIShort Range green1650fps 50 cartouches CCI 22LR short range green 50 cartouches de CCI short range green 22LR de 21 grains d'une vélocité de 1650fps en Hollow point adapté au tir sur cible plinking. Appréciée pour leur rapidité, elles ne contiennent pas de plombs, c'est pourquoi elles sont dites vertes. La charge sans plombs est respectueuse de l'environnement, idéal pour expulser la vermine et autre petits gibiers ainsi que le tir sur cible. Elle dispose d'une balle faite à partir d'un mélange de cuivre et de particules polymères. Caractéristiques des cartouches CCI Short Range Green Calibre 22LR pour carabine long rifle CCI Short Range Green Légères et robustes Conçues pour le tir longue distance Plinking",20.58,50,50
437,Balles Norma Eco Power - Cal. 22LR - 22LR / Par 1 / 24,https://www.naturabuy.fr/Balles-Norma-Eco-Power-Cal-22LR-Par-1-item-9176087.html,Norma,1,10.80,9.90,"Marque : NormaEtat de l'objet : NeufType : StandardType d'ogive : Cuivre Norma ECO propose des projectiles sans plomb avec une balistique et un effet excellents. Les balles de cette ligne sont des projectiles en zinc plaqué cuivre. Caractéristiques : - Sans plomb- Balle zinc plaqué cuivre- Boite de 50 balles- Calibre : 22LR- Ogive : Eco Power Quantité: Par 1, par 10 ou par 20 (voir options) Calibre22LRGrammes1.6 gGrains24Vitesse à 0 m520 m/sVitesse à 100 m313 m/sVitesse à 200 m250 m/sEnergie à 0 m210 joulesEnergie à 100 m76 joulesEnergie à 200 m48 joulesEnergie à 300 m34 joules L'expédition de votre commande ne sera pas effectuée tant que la totalité des justificatifs ne seront pas réceptionnés et validés.",20.70,0,50
484,MUNITIONS AGUILA .22 SHORT 29G X50,https://www.naturabuy.fr/MUNITIONS-AGUILA-22-SHORT-29G-X50-item-8953374.html,Aguila,1,9.20,16.08,Marque : AguilaEtat de l'objet : NeufType : StandardType d'ogive : Cuivre Coté technique: Calibre 22 short Aguila Boite de 50 Ogive cuivré de 29 grains AGUILA Super Extra Short 29 grains Balle pleine cuivrée spéciale tir sur cible. Vitesse 1095 fps (333 m/s) Projectile cuivré pour limiter l'encrassement du canon. Boites de 50 cartouches Munitions Catégorie C 8° Acquisition sur présentation de la licence de tir en cours de validité avec cachet du médecin ou permis de chasser avec validation annuelle de l'année en cours ou de l'année précédente.,25.28,0,50


In [ ]:
# get stats on results

# Calculate counts and percentages of zeros in 'ammo_qtty_title'
title_zero_count = df_v1['ammo_qtty_title'].value_counts().get(0, 0)  # Get the count of zeros
title_total_count = len(df_v1)  # Total number of rows in the column
title_zero_percent = (title_zero_count / title_total_count) * 100

# Calculate counts and percentages of zeros in 'ammo_qtty_description'
descr_zero_count = df_v1['ammo_qtty_description'].value_counts().get(0, 0)  # Get the count of zeros
descr_total_count = len(df_v1)  # Total number of rows in the column
descr_zero_percent = (descr_zero_count / descr_total_count) * 100

print(f"Count of zeros in 'ammo_qtty_title': {title_zero_count}")
print(f"Percentage of zeros in 'ammo_qtty_title': {title_zero_percent:.2f}%")

print(f"Count of zeros in 'ammo_qtty_description': {descr_zero_count}")
print(f"Percentage of zeros in 'ammo_qtty_description': {descr_zero_percent:.2f}%")

### Search all 
new way 2

In [ ]:
# read back raw data from csv
df = pd.read_csv('scraped_data_clean.csv')

In [28]:
# detects number divisible by 50 w\o remainder, and 525 and 325
# 0 value if nothing was found
# STOPS @ last

def find_all_ammo_quantities(text):
    pattern = r'\b(50|525|325)\b'  # Regular expression pattern to match required numbers
    matches = re.findall(pattern, text)
    return [int(match) for match in matches]

"""def process_all_matches(df):
    df['ammo_qtty_title'] = df['product_name'].apply(find_all_ammo_quantities)
    df['ammo_qtty_description'] = df['product_description'].apply(find_all_ammo_quantities)
    return df

df_v2 = process_all_matches(df)"""

def process_all_matches(df):
    df['ammo_qtty_title'] = df['product_name'].apply(find_all_ammo_quantities)
    df['ammo_qtty_description'] = df['product_description'].apply(find_all_ammo_quantities)
    
    df['empty_ammo_qtty_title'] = df['ammo_qtty_title'].apply(lambda x: len(x) == 0) # for counting only
    df['empty_ammo_qtty_description'] = df['ammo_qtty_description'].apply(lambda x: len(x) == 0) # for counting only
    
    return df

df_v2 = process_all_matches(df)

# Count empty cases
empty_ammo_qtty_title_count = df_v2['empty_ammo_qtty_title'].sum()
empty_ammo_qtty_description_count = df_v2['empty_ammo_qtty_description'].sum()

# Calculate percentages
total_rows = len(df_v2)
empty_ammo_qtty_title_percent = (empty_ammo_qtty_title_count / total_rows) * 100
empty_ammo_qtty_description_percent = (empty_ammo_qtty_description_count / total_rows) * 100

print(f"Empty 'ammo_qtty_title' count: {empty_ammo_qtty_title_count}")
print(f"Empty 'ammo_qtty_description' count: {empty_ammo_qtty_description_count}")

print(f"Percentage of empty 'ammo_qtty_title': {empty_ammo_qtty_title_percent:.2f}%")
print(f"Percentage of empty 'ammo_qtty_description': {empty_ammo_qtty_description_percent:.2f}%")

Empty 'ammo_qtty_title' count: 1004
Empty 'ammo_qtty_description' count: 428
Percentage of empty 'ammo_qtty_title': 81.23%
Percentage of empty 'ammo_qtty_description': 34.63%


In [26]:
df_v2.sample(5)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title,ammo_qtty_description,empty_ammo_qtty_title,empty_ammo_qtty_description
1164,50 Munitions NORMA Cal 22lr 40gr Subsonic,https://www.naturabuy.fr/50-MUNITIONS-NORMA-CAL-22LR-40GR-SUBSONIC-item-7826633.html,Norma,1,8.0,15.0,Marque : NormaEtat de l'objet : NeufType : Subsonique Boite de 20 balles Norma. Produit peut être disponible sur un autre support MERCI DE NOUS CONSULTER POUR VERIFIER L'ETAT DE STOCK,23.0,[50],[],False,True
85,RWS 22 LR P25 Rapid Fire 2.6g 40gr x1 boite,https://www.naturabuy.fr/RWS-22-LR-P25-Rapid-Fire-2-6g-40gr-x1-boite-item-9861209.html,RWS,1,26.8,8.0,Marque : RWSEtat de l'objet : NeufType d'ogive : Plomb !!! AFFAIRE A SAISIR !! ATS-46 vous propose ces munitions 22 LR P25 Rapid Fire de la marque RWS. CARACTERISTIQUES Marque : RWS Calibre : 22 LR P25 Rapid Fire Ogive : Plomb Poids : 2.6 g Grains : 40 gr Munition / boite : 50 NEUF X1 BOITE ATS-46 vous informe que les munitions en stock magasin vous seront envoyées sous 48H et 96H pour les commandes directement au fournisseur.,34.8,[],[50],True,False
1173,50 Munitions FEDERAL American Eagle Subsonic Cal 22Lr 45Gr,https://www.naturabuy.fr/50-Munitions-FEDERAL-American-Eagle-Subsonic-Cal-22Lr-45Gr-item-9340592.html,Federal,1,10.5,12.0,Marque : FederalEtat de l'objet : NeufType : SubsoniqueType d'ogive : Cuivre Boite de 20 Munitions FEDERAL American Eagle Subsonic Calibre 22Lr 45Gr Produit peut être vendu sur un autre support MERCI DE NOUS CONSULTER POUR VERIFIER L'ETAT DE STOCK,22.5,[50],[],False,True
1040,BALLES NORMA CAL.22LR BIATHLON SPORT-22 2.6G 40GR PAR 50,https://www.naturabuy.fr/BALLES-NORMA-CAL-22LR-BIATHLON-SPORT-22-2-6G-40GR-PAR-50-item-10691444.html,Norma,1,15.8,9.9,"Marque : NormaEtat de l'objet : NeufType : StandardType d'ogive : Plomb BALLES NORMA CAL.22LR BIATHLON SPORT-22 2.6G 40GR PAR 50 Coefficient balistique G1 0,135",25.7,[50],[50],False,False
455,Balles Norma Eco Power - Cal. 22LR - 22LR / Par 10 / 24,https://www.naturabuy.fr/Balles-Norma-Eco-Power-Cal-22LR-Par-10-item-9273390.html,Norma,1,108.0,9.9,"Marque : NormaEtat de l'objet : NeufType : StandardType d'ogive : Cuivre Norma ECO propose des projectiles sans plomb avec une balistique et un effet excellents. Les balles de cette ligne sont des projectiles en zinc plaqué cuivre. Caractéristiques : - Sans plomb- Balle zinc plaqué cuivre- Boite de 50 balles- Calibre : 22LR- Ogive : Eco Power Quantité: Par 1, par 10 ou par 20 (voir options) Calibre22LRGrammes1.6 gGrains24Vitesse à 0 m520 m/sVitesse à 100 m313 m/sVitesse à 200 m250 m/sEnergie à 0 m210 joulesEnergie à 100 m76 joulesEnergie à 200 m48 joulesEnergie à 300 m34 joules L'expédition de votre commande ne sera pas effectuée tant que la totalité des justificatifs ne seront pas réceptionnés et validés.",117.9,[],[50],True,False


### new way 3

In [30]:
# read back raw data from csv
df = pd.read_csv('scraped_data_clean.csv')

In [31]:
# detects number divisible by 50 w\o remainder, and 525 and 325
# 0 value if nothing was found
# STOPS @ 1st

def find_ammo_quantity(text):
    pattern = r'\b(?:50|[35]25)\b'  # Regular expression pattern to match 50, 525, or 325
    numbers = [int(match) for match in re.findall(pattern, text)]
    return numbers[0] if numbers else 0

def compare_first_numbers(num1, num2):
    return int(num1 == num2)

def solution_a(df):
    df['ammo_qtty_title'] = df['product_name'].apply(find_ammo_quantity)
    df['ammo_qtty_description'] = df['product_description'].apply(find_ammo_quantity)
    df['qtty_titledescr_equal'] = df.apply(lambda row: compare_first_numbers(row['ammo_qtty_title'], row['ammo_qtty_description']), axis=1)
    return df

# Usage example:
df_v11 = solution_a(df)

In [13]:
df_v11.sample(10)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title,ammo_qtty_description,qtty_titledescr_equal
1137,Cartouches 22 LR Aguila Subsonic Hollow Point,https://www.naturabuy.fr/Cartouches-22-LR-Aguila-Subsonic-Hollow-Point-item-7831722.html,Aguila,1,118.00,14.90,Marque : AguilaEtat de l'objet : NeufType : SubsoniqueType d'ogive : Creuse Ogive 38 grains subsonique à tête creuse. Cartouche vélocité standard : 1024 fps (312 m/s) soit une énergie de 120 joules. 10 boîtes de 50 soit 1 carton de 500.Détails techniques : Législation pour la vente : Catégorie C - Présentation permis de chasse + validation OU licence de tir OU licence de ball-trap (Plus de détails dans législation)Calibre : .22 LRCdt : 500Énergie (J) : 120Grains : 38Grammes : 2.46Vitesse (m/s) : 312,132.90,0,50,0
360,Balles RWS R100 - Cal. 22LR - 22LR / Par 1 / 40,https://www.naturabuy.fr/Balles-RWS-R100-Cal-22LR-Par-1-item-9179470.html,RWS,1,25.00,9.90,"Marque : RWSEtat de l'objet : NeufType : StandardType d'ogive : Plomb Munitions RWS Cal.22lr Premium Line R100La munition RWS R 100 est la préférée des tireurs sportifs de compétition au niveau international. Vitesse supersonique et précision remarquable font de la RWS R 100 une munition de premier choix pour le tir sur silhouettes à l'arme d'épaule et de poing.Adaptée au tir à 50 et 100 m.Merci de vous renseigner auprès de vos instances quant à l'utilisation de munitions à percussion annulaire RWS .22lr R100 en biathlon.Le poids de la balle est exactement de 2.675 +/- 0.005 grammes Quantité: Par 1, par 10 ou par 20 (voir options)",34.90,0,50,0
287,Balles Federal Premium Match - Cal. 22 LR - Par 1 / 22LR / 40,https://www.naturabuy.fr/Balles-Federal-Premium-Match-Cal-22-LR-Par-1-item-9207325.html,Federal,1,31.25,9.90,"Marque : FederalEtat de l'objet : NeufType : MatchType d'ogive : Plomb Utilisez le meilleur pour la compétition. Ce sont les chargements d'usine les plus précis. Et c'est la technologie dont vous avez besoin pour ramener quelques médailles à la maison. /Caractéristiques: Calibre 22 LRBoîte50OgiveSolid100ms277Poids2.59Joules 100m100Joules 50m117Grains40 Quantité: Par 1, par 10 ou par 20 (voir options)",41.15,0,0,1
608,CCI STINGER,https://www.naturabuy.fr/CCI-STINGER-item-10013678.html,CCI,1,15.00,9.00,Marque : CCIEtat de l'objet : NeufType d'ogive : Creuse MUNITIONS CCI HAUTE VITESSE AVEC BALLE DE 32 GRS A 499 M-S MUNITIONS DITES DE VARMINT TRES HAUT POUVOIR PENETRANT ET EXPENSIF A MON AVIS LA MUNITION 22LR LA PLU PUISSANTE DU MARCHE,24.00,0,0,1
21,RWS 22 LR High Velocity Green 1.6g 24gr x10 boites,https://www.naturabuy.fr/RWS-22-LR-High-Velocity-Green-1-6g-24gr-x10-boites-item-9861604.html,RWS,1,134.00,0.00,Marque : RWSEtat de l'objet : Neuf !!! AFFAIRE A SAISIR !! ATS-46 vous propose ces munitions 22 LR High Velocity Green de la marque RWS. CARACTERISTIQUES Marque : RWS Calibre : 22 LR High Velocity Green Ogive : Zinc Poids : 1.6 g Grains : 24 gr Munition / boite : 50 NEUF X10 BOITES Frais de port offert ! profitez-en ! ATS-46 vous informe que les munitions en stock magasin vous seront envoyées sous 48H et 96H pour les commandes directement au fournisseur.,134.00,0,50,0
232,Balles Norma RIMF CTG Xtreme - Cal. 22 LR - 2.8 g,https://www.naturabuy.fr/Balles-Norma-RIMF-CTG-Xtreme-Cal-22-LR-2-8-g-item-10076550.html,Norma,1,20.40,9.90,"Marque : NormaEtat de l'objet : NeufType : StandardType d'ogive : Plomb Caractéristiques : - Spécialement développée pour le tir longue distance (>200m)- Conception unique avec coeff. balistique élevé- Trajectoire tendue- Poids élevé pour une meilleure trajectoire- Recommandé pour les concours de tir longue distance en percussion annulaire- Calibre : .22 long pour carabine- Projectile de plomb de 2,8g, V0 355 m/s (longueur de canon : 65 cm)- Boite de 50",30.30,0,50,0
428,Balles RWS Subsonic Hp - Cal. 22LR - Par 20,https://www.naturabuy.fr/Balles-RWS-Subsonic-Hp-Cal-22LR-Par-20-item-9273642.html,RWS,1,182.00,9.90,"Marque : 

In [32]:
# get stats on results

# Calculate counts and percentages of zeros in 'ammo_qtty_title'
title_zero_count = df_v11['ammo_qtty_title'].value_counts().get(0, 0)  # Get the count of zeros
title_total_count = len(df_v11)  # Total number of rows in the column
title_zero_percent = (title_zero_count / title_total_count) * 100

# Calculate counts and percentages of zeros in 'ammo_qtty_description'
descr_zero_count = df_v11['ammo_qtty_description'].value_counts().get(0, 0)  # Get the count of zeros
descr_total_count = len(df_v11)  # Total number of rows in the column
descr_zero_percent = (descr_zero_count / descr_total_count) * 100

print(f"Count of zeros in 'ammo_qtty_title': {title_zero_count}")
print(f"Percentage of zeros in 'ammo_qtty_title': {title_zero_percent:.2f}%")

print(f"Count of zeros in 'ammo_qtty_description': {descr_zero_count}")
print(f"Percentage of zeros in 'ammo_qtty_description': {descr_zero_percent:.2f}%")

Count of zeros in 'ammo_qtty_title': 1004
Percentage of zeros in 'ammo_qtty_title': 81.23%
Count of zeros in 'ammo_qtty_description': 428
Percentage of zeros in 'ammo_qtty_description': 34.63%


### new way 4

In [33]:
# read back raw data from csv
df = pd.read_csv('scraped_data_clean.csv')

In [34]:
def find_matching_numbers(text):
    pattern = r'\b(\d+)\b'
    numbers = re.findall(pattern, text)
    matching_numbers = [int(num) for num in numbers if int(num) % 50 == 0 or int(num) in [325, 525]]
    return matching_numbers

def solution_a(df):
    df['ammo_qtty_title'] = df['product_name'].apply(lambda x: find_matching_numbers(x))
    df['ammo_qtty_description'] = df['product_description'].apply(lambda x: find_matching_numbers(x))

    # Find the first match in each column and store in new column 'qtty_equal'
    df['qtty_equal'] = (df['ammo_qtty_title'].apply(lambda x: x[0] if len(x) > 0 else 0) ==
                        df['ammo_qtty_description'].apply(lambda x: x[0] if len(x) > 0 else 0)).astype(int)

    # Fill any missing values with 0
    df['ammo_qtty_title'] = df['ammo_qtty_title'].apply(lambda x: x[0] if len(x) > 0 else 0)
    df['ammo_qtty_description'] = df['ammo_qtty_description'].apply(lambda x: x[0] if len(x) > 0 else 0)

    return df

def solution_b(df):
    df['ammo_qtty_title'] = df['product_name'].apply(lambda x: find_matching_numbers(x))
    df['ammo_qtty_description'] = df['product_description'].apply(lambda x: find_matching_numbers(x))

    # New columns to check if all numbers are equal in 'ammo_qtty_title' and 'ammo_qtty_description'
    df['qtty_title_equal'] = df['ammo_qtty_title'].apply(lambda x: int(len(set(x)) == 1))  # 1 if all numbers are equal, 0 otherwise
    df['qtty_titledescr_equal'] = df['ammo_qtty_description'].apply(lambda x: int(len(set(x)) == 1))  # 1 if all numbers are equal, 0 otherwise

    return df

In [35]:
# Apply Solution A and Solution B
df_solution_a = solution_a(df.copy())
df_solution_b = solution_b(df.copy())

In [25]:
df.head()

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price
0,BOITE DE 325 CARTOUCHES 22LR FEDERAL GOLD PLATED HOLLOW POINT 36 GRAINS,https://www.naturabuy.fr/BOITE-DE-325-CARTOUCHES-22LR-FEDERAL-GOLD-PLATED-HOLLOW-POINT-36-GRAINS-item-10619103.html,Federal,1,46.9,12.0,Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Creuse boite de 325 cartouches de marque Federal 22lr 36 grains round nose hollow point cuivree excellente cartouche pour le tir dans les armes a verrou comme semi automatique marque : Fédéral référence : 725 Type d’ogive round nose cuivree a pointe creuse vitesse 384 mètres par secondes poids ogive : 36 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation,58.9
1,Boîte ancienne 22 LR Remington 22 Target,https://www.naturabuy.fr/Boite-ancienne-22-LR-Remington-22-Target-item-10653966.html,Remington,0,11.0,4.4,Marque : RemingtonEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte en parfait état. Envoie par mondial relais 4€40,15.4
2,"boite de 325 cartouches de 22Lr de marque Federal , Automatch , pour semi auto",https://www.naturabuy.fr/boite-325-cartouches-22Lr-marque-Federal-Automatch-semi-auto-item-10619099.html,Federal,1,44.9,12.0,"Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Plomb 1 boite de 325 cartouches de marque Federal 22lr ,40 grains round nose special semi auto Type d'ogive round nose en plomb vitesse 360 metres par secondes poids ogive : 40 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation",56.9
3,Boîte ancienne 22 LR Fiocchi expansive,https://www.naturabuy.fr/Boite-ancienne-22-LR-Fiocchi-expansive-item-10653962.html,Fiocchi,0,12.0,4.4,"Marque : FiocchiEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte ancienne, Fiocchi expansive, il manque 6 cartouches. Envoie par mondial relais 4,40 €",16.4
4,Boîte ancienne 22 LR NCS 20 Match,https://www.naturabuy.fr/Boite-ancienne-22-LR-NCS-20-Match-item-10653952.html,NCS,0,20.0,4.4,"Marque : NCSEtat de l'objet : D'occasionType : MatchType d'ogive : Plomb Boîte ancienne frabriquée par la Nouvelle Cartoucherie de Survilliers (Cartoucherie Française). Boîte en parfait état. Envoie par mondial relay 4,40 €",24.4


In [23]:
df_solution_a.sample(10)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title,ammo_qtty_description,qtty_equal
0,BOITE DE 325 CARTOUCHES 22LR FEDERAL GOLD PLATED HOLLOW POINT 36 GRAINS,https://www.naturabuy.fr/BOITE-DE-325-CARTOUCHES-22LR-FEDERAL-GOLD-PLATED-HOLLOW-POINT-36-GRAINS-item-10619103.html,Federal,1,46.9,12.0,Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Creuse boite de 325 cartouches de marque Federal 22lr 36 grains round nose hollow point cuivree excellente cartouche pour le tir dans les armes a verrou comme semi automatique marque : Fédéral référence : 725 Type d’ogive round nose cuivree a pointe creuse vitesse 384 mètres par secondes poids ogive : 36 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation,58.9,325,325,1
1,Boîte ancienne 22 LR Remington 22 Target,https://www.naturabuy.fr/Boite-ancienne-22-LR-Remington-22-Target-item-10653966.html,Remington,0,11.0,4.4,Marque : RemingtonEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte en parfait état. Envoie par mondial relais 4€40,15.4,0,0,1
2,"boite de 325 cartouches de 22Lr de marque Federal , Automatch , pour semi auto",https://www.naturabuy.fr/boite-325-cartouches-22Lr-marque-Federal-Automatch-semi-auto-item-10619099.html,Federal,1,44.9,12.0,"Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Plomb 1 boite de 325 cartouches de marque Federal 22lr ,40 grains round nose special semi auto Type d'ogive round nose en plomb vitesse 360 metres par secondes poids ogive : 40 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation",56.9,325,325,1
3,Boîte ancienne 22 LR Fiocchi expansive,https://www.naturabuy.fr/Boite-ancienne-22-LR-Fiocchi-expansive-item-10653962.html,Fiocchi,0,12.0,4.4,"Marque : FiocchiEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte ancienne, Fiocchi expansive, il manque 6 cartouches. Envoie par mondial relais 4,40 €",16.4,0,0,1
4,Boîte ancienne 22 LR NCS 20 Match,https://www.naturabuy.fr/Boite-ancienne-22-LR-NCS-20-Match-item-10653952.html,NCS,0,20.0,4.4,"Marque : NCSEtat de l'objet : D'occasionType : MatchType d'ogive : Plomb Boîte ancienne frabriquée par la Nouvelle Cartoucherie de Survilliers (Cartoucherie Française). Boîte en parfait état. Envoie par mondial relay 4,40 €",24.4,0,0,1
5,Boîte 22 LR ancienne NCS 30,https://www.naturabuy.fr/Boite-22-LR-ancienne-NCS-30-item-10653948.html,NCS,0,20.0,4.4,"Marque : NCSEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte ancienne, fabrication Nouvelle Cartoucherie de Survilliers (Cartouche rie française). Bôite complète en en parfait état. Envoie par mondial relais 4,40 €",24.4,0,0,1
6,Boîte ancienne 22 LR Fiocchi ultrasonic,https://www.naturabuy.fr/Boite-ancienne-22-LR-Fiocchi-ultrasonic-item-10653942.html,Fiocchi,0,15.0,4.4,"Marque : FiocchiEtat de l'objet : D'occasionType : Standard Longue - haute vitesseType d'ogive : Cuivre Boîte ancienne Fiocchi Ultrasonic, boîte en parfait état. Envoie par mondial relais 4 € 40",19.4,0,0,1
7,Boite ancienne T22,https://www.naturabuy.fr/Boite-ancienne-T22-item-10270875.html,Winchester,0,10.0,4.4,Marque : WinchesterEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte complète de Winchester T22. Dernier exemplaire Envoi par Mondial Relais 4€40,14.4,0,0,1
8,MUNITIONS SUBSONIC CAL. 22 LR boite de 50 balles,https://www.naturabuy.fr/MUNITIONS-SUBSONIC-CAL-22-LR-item-9413972.html,Winchester,1,18.9,8.6,Marque : WinchesterEtat de l'objet : NeufType : SubsoniqueType d'ogive : Creuse Ogives en plomb 42 grains. Profil Hollow Point (Pointe creuse) expansive,27.5,50,0,0
9,22 lr bosquette x 50 en vrac conique,https://www.naturabuy.fr/22-lr-bosquette-100-vrac-conique-item-10385603.html,NCS,1,1.0,9.0,Marque : NCSEtat de l'obj

In [26]:
df_solution_b.sample(10)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title,ammo_qtty_description,qtty_title_equal,qtty_titledescr_equal
219,Cartouches 22LR RWS PISTOL MATCH par 500,https://www.naturabuy.fr/Cartouches-22LR-RWS-PISTOL-MATCH-par-500-item-10709133.html,RWS,1,86.0,14.00,Marque : RWSEtat de l'objet : NeufType : MatchType d'ogive : Plomb Cartouches 22LR RWS PISTOL MATCH -par 500 !!!,100.00,[500],[500],1,1
238,Balles Winchester T22 - Cal. 22LR - 40 gr LRN - 22LR / Par 1 / 40,https://www.naturabuy.fr/Balles-Winchester-T22-Cal-22LR-40-gr-LRN-Par-1-item-9207515.html,Winchester,1,7.5,9.90,"Marque : WinchesterEtat de l'objet : NeufType : StandardType d'ogive : Plomb - Boîte de 50 balles- Calibre : 22LR- Ogive : Target Quantité: Par 1, par 10 ou par 20 (voir options) Calibre22LRGrains40Vitesse à 0 m331 m/s L'expédition de votre commande ne sera pas effectuée tant que la totalité des justificatifs ne seront pas réceptionnés et validés.",17.40,[],"[50, 0]",0,0
835,STINGER - CCI,https://www.naturabuy.fr/STINGER-CCI-item-7316795.html,CCI,1,20.5,30.12,"Marque : CCIEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Creuse Type: CARTOUCHE - Marque: CCI - Modèle: STINGER - Calibre: .22 LR - Type de balle: CPHP - Ogive: Pointe creuse cuivrée - Poids grain (GR): 32 GR - Poids (g): 2,07 g - V0 (m/s): 500 - V50 (m/s): 394 - V100 (m/s): 325 - E0 (Joules): 259 - E50 (Joules): 161 - E100 (Joules): 110 - Conditionnement: BOÎTE DE 50 - Catégorie : Catégorie C",50.62,[],"[500, 325, 50]",0,0
607,".22 court, Target (Calibre: .22 court)",https://www.naturabuy.fr/-22-court-Target-Calibre-22-court--item-9185512.html,Inconnue,1,7.4,14.90,"Marque : InconnueEtat de l'objet : NeufType : StandardType d'ogive : Plomb Cartouche à percussion annulaire, .22 courte, target. Par boite = 50 cartouches. Prix quantitatif pour les munitions Sellier & Bellot sur demande.",22.30,[],[50],0,1
918,22lr RWS Biathlon Special Match,https://www.naturabuy.fr/22lr-RWS-Biathlon-Special-Match-item-8769332.html,Norma,1,15.0,4.95,"Marque : NormaEtat de l'objet : NeufType : MatchType d'ogive : Plomb Boite de 50 cartouches 22LR Marque RWS Biathlon Special Match Ogives : 40 grains (2.6grammes) , plomb , round nose Vitesse: 1050 fps (320 m/ s) Energie : 137 joules Utilisation : biathlon , jusqu'à -20°C en stock",19.95,[],"[50, 1050]",0,0
100,RWS 22 LR R50 2.6g 40gr x1 boite,https://www.naturabuy.fr/RWS-22-LR-R50-2-6g-40gr-x1-boite-item-9861174.html,RWS,1,26.8,8.00,Marque : RWSEtat de l'objet : NeufType d'ogive : Plomb !!! AFFAIRE A SAISIR !! ATS-46 vous propose ces munitions 22 LR R50 de la marque RWS. CARACTERISTIQUES Marque : RWS Calibre : 22 LR R50 Ogive : plomb Poids : 2.6 g Grains : 40 gr Munition / boite : 50 NEUF X1 BOITE ATS-46 vous informe que les munitions en stock magasin vous seront envoyées sous 48H et 96H pour les commandes directement au fournisseur.,34.80,[],[50],0,1
624,BOITE DE 100 CARTOUCHES CCI 22 LR GRENNTAG PRECISION HAUTE VITESSE COMPETITION 1070 FPS 40 GR,https://www.naturabuy.fr/BOITE-DE-100-CARTOUCHES-CCI-22-LR-GRENNTAG-PRECISION-HAUTE-VITESSE-COMPETITION-1070-FPS-40-GR-item-9727987.html,CCI,1,22.0,6.50,"Marque : CCIEtat de l'objet : NeufType : Match Bonjour, Je vends une boîte neuve de 100 cartouches CCI MATCH Haute Vitesse 1070 FPS GRENNTAG PRECISION 40 Grains 100 Cartouches 22LR LEAD ROUND NOSE Compétition Haute Vitesse 1070 FPS Cordialement",28.50,[100],"[100, 100]",1,1
40,RWS 22 LR Target Pistol 2.6g 40gr x1 boite,https://www.naturabuy.fr/RWS-22-LR-Target-Pistol-2-6g-40gr-x1-boite-item-9861417.html,RWS,1,8.9,8.00,Marque : RWSEtat de l'objet : NeufType d'ogive : Plomb !!! AFFAIRE A SAISIR !! ATS-46 vous propose ces munitions 22 LR Target Pistol de la marque RWS. CARACTERISTIQUES Marque : RWS Calibre : 22 LR Target Pistol Ogive : Plomb Poids : 2.6 g Grains : 40 gr Munition / boite : 50 NEUF X1 BOITE ATS-46 vous informe que les munitions en stock magasin vous seront envoyées sous 48H et 96

In [36]:
# get stats on results

# Calculate counts and percentages of zeros in 'ammo_qtty_title'
title_zero_count = df_solution_a['ammo_qtty_title'].value_counts().get(0, 0)  # Get the count of zeros
title_total_count = len(df_solution_a)  # Total number of rows in the column
title_zero_percent = (title_zero_count / title_total_count) * 100

# Calculate counts and percentages of zeros in 'ammo_qtty_description'
descr_zero_count = df_solution_a['ammo_qtty_description'].value_counts().get(0, 0)  # Get the count of zeros
descr_total_count = len(df_solution_a)  # Total number of rows in the column
descr_zero_percent = (descr_zero_count / descr_total_count) * 100

print(f"Count of zeros in 'ammo_qtty_title': {title_zero_count}")
print(f"Percentage of zeros in 'ammo_qtty_title': {title_zero_percent:.2f}%")

print(f"Count of zeros in 'ammo_qtty_description': {descr_zero_count}")
print(f"Percentage of zeros in 'ammo_qtty_description': {descr_zero_percent:.2f}%")

Count of zeros in 'ammo_qtty_title': 685
Percentage of zeros in 'ammo_qtty_title': 55.42%
Count of zeros in 'ammo_qtty_description': 251
Percentage of zeros in 'ammo_qtty_description': 20.31%


In [37]:
# get stats on results

# Calculate counts and percentages of zeros in 'ammo_qtty_title'
title_zero_count = df_solution_b['ammo_qtty_title'].value_counts().get(0, 0)  # Get the count of zeros
title_total_count = len(df_solution_b)  # Total number of rows in the column
title_zero_percent = (title_zero_count / title_total_count) * 100

# Calculate counts and percentages of zeros in 'ammo_qtty_description'
descr_zero_count = df_solution_b['ammo_qtty_description'].value_counts().get(0, 0)  # Get the count of zeros
descr_total_count = len(df_solution_b)  # Total number of rows in the column
descr_zero_percent = (descr_zero_count / descr_total_count) * 100

print(f"Count of zeros in 'ammo_qtty_title': {title_zero_count}")
print(f"Percentage of zeros in 'ammo_qtty_title': {title_zero_percent:.2f}%")

print(f"Count of zeros in 'ammo_qtty_description': {descr_zero_count}")
print(f"Percentage of zeros in 'ammo_qtty_description': {descr_zero_percent:.2f}%")

Count of zeros in 'ammo_qtty_title': 685
Percentage of zeros in 'ammo_qtty_title': 55.42%
Count of zeros in 'ammo_qtty_description': 382
Percentage of zeros in 'ammo_qtty_description': 30.91%


### old solution 1

In [59]:
# read back raw data from csv
df = pd.read_csv('scraped_data_clean.csv')

In [60]:
# regex to catch any number divisible by 50 (min qtty of rounds in a box of ammo)
def extract_bullet_qtty(text):
    # match any number that is divisible by 50 without remainder
    regex = r"\b(0|[5-9]\d*[0]|100)\s*(?:boites de\s*)?(?:cartouches|balles|munitions)\b"
    match = re.search(regex, text, re.IGNORECASE)
    if match:
        # extract the matched number and convert it to integer
        qtty = int(match.group(1))
        # round the quantity to the nearest 50
        qtty = (qtty // 50) * 50
        return qtty
    else:
        return None
#
# check titles with regex
df['bullet_qtty'] = df['product_name'].apply(extract_bullet_qtty)
#
# check for empty cells, if any do a pass of regex on product description --- !!! DUPE avoidance !!!
df.loc[df['bullet_qtty'].isnull(), 'bullet_qtty'] = df['product_description'].apply(extract_bullet_qtty)
#
# if still no data, we fill with 50 for default min number of ammo per box
#df['bullet_qtty'].fillna(50, inplace=True)

In [61]:
df.sample(3)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,bullet_qtty
105,RWS 22 LR Biathlon Compétition 2.6g 40gr x10 boites,https://www.naturabuy.fr/RWS-22-LR-Biathlon-Competition-2-6g-40gr-x10-boites-item-9861112.html,RWS,1,280.0,0.0,Marque : RWSEtat de l'objet : NeufType d'ogive : Plomb !!! AFFAIRE A SAISIR !! ATS-46 vous propose ces munitions 22 LR Biathlon Compétition de la marque RWS. CARACTERISTIQUES Marque : RWS Calibre : 22 LR Biathlon Compétition Ogive : plomb Poids : 2.6 g Grains : 40 gr Munition / boite : 50 NEUF X10 BOITES Frais de port offert ! profitez-en ! ATS-46 vous informe que les munitions en stock magasin vous seront envoyées sous 48H et 96H pour les commandes directement au fournisseur.,280.0,NaN
1088,Boîte ancienne 22 LR Gévelot GP (2),https://www.naturabuy.fr/Boite-ancienne-22-LR-Gevelot-GP-2--item-10655144.html,Gévelot,0,15.0,4.4,Marque : GévelotEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte Gévelot GP Envoie par mondial relais 4€40,19.4,NaN
832,Munitions RWS Cal.22lr Field Line High Velocity par 500,https://www.naturabuy.fr/Munitions-RWS-Cal-22lr-Field-Line-High-Velocity-par-500-item-9634203.html,RWS,1,87.0,12.0,Marque : RWSEtat de l'objet : Neuf munitions RWS Cal.22lr Field Line High Velocity La munition RWS HIGH VELOCITY (= haute vitesse) a été conçue pour la chasse des gibiers de petite taille. Sa vitesse plus élevée - elle affiche une différence de plus 80 m/s - donne à cette munition de petit calibre une efficacité particulière. Elle est particulièrement adaptée aux armes d'épaule et aux réducteurs de canons. Cette munition est équipée d'une balle de plomb à chemise cuivrée. Boite de 50 500 munitions soit 10 boites de 50,99.0,500.0


In [62]:
# get stats on results

# Calculate counts and percentages of zeros in 'bullet_qtty'
title_zero_count = df['bullet_qtty'].value_counts().get(0, 0)  # Get the count of zeros
title_total_count = len(df)  # Total number of rows in the column
title_zero_percent = (title_zero_count / title_total_count) * 100

print(f"Count of zeros in 'bullet_qtty': {title_zero_count}")
print(f"Percentage of zeros in 'bullet_qtty': {title_zero_percent:.2f}%")

Count of zeros in 'bullet_qtty': 0
Percentage of zeros in 'bullet_qtty': 0.00%


### old solution 2
search for all occurences in TITLE.
for each match create new col and store value

In [63]:
# read back raw data from csv
df = pd.read_csv('scraped_data_clean.csv')

In [64]:
# qtty of ammo search in TITLE

# Regular expression pattern for finding numbers divisible by 50 without remainder
pattern1 = r'\b(?:[5-9]|[1-9]\d{1,})0\b|\b525\b'

# Function to find matching numbers and store in new columns
def find_ammo_qtty_title(df):
    # Find all matching numbers and store them in a list
    matches = df['product_name'].apply(lambda x: re.findall(pattern1, str(x)))
    
    # Create new columns for each match and store the results
    for i, match in enumerate(matches):
        for j in range(len(match)):
            column_name = f'ammo_qtty_title_{j+1}'
            df.at[i, column_name] = match[j]
    
    return df

In [65]:
df_old2 = find_ammo_qtty_title(df)

df_old2.sample(1)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title_1,ammo_qtty_title_2,ammo_qtty_title_3,ammo_qtty_title_4
445,Balles Winchester Super-X CP - Cal. 22LR - 22LR / Par 10,https://www.naturabuy.fr/Balles-Winchester-Super-X-CP-Cal-22LR-Par-10-item-9273113.html,Winchester,1,182.0,9.9,"Marque : WinchesterEtat de l'objet : NeufType : StandardType d'ogive : Cuivre - Boîte de 100 balles- Calibre : 22LR- Ogive : Super-X Quantité: Par 1, par 10 ou par 20 (voir options) Calibre22LRGrains37Grammes2.40 gVitesse à 0 m405 m/sVitesse à 50 m316 m/sEnergie à 0 m197 joulesEnergie à 50 m119 joules L'expédition de votre commande ne sera pas effectuée tant que la totalité des justificatifs ne seront pas réceptionnés et validés.",191.9,NaN,NaN,NaN,NaN


In [66]:
# qtty of ammo search in DESCRIPTION

# Regular expression pattern for finding numbers divisible by 50 without remainder
pattern2 = r'\b(?:[5-9]|[1-9]\d{1,})0\b|\b525\b'

# Function to find matching numbers and store in new columns
def find_ammo_qtty_description(df):
    # Find all matching numbers and store them in a list
    matches = df['product_description'].apply(lambda x: re.findall(pattern2, str(x)))
    
    # Create new columns for each match and store the results
    for i, match in enumerate(matches):
        for j in range(len(match)):
            column_name = f'ammo_qtty_description_{j+1}'
            df.at[i, column_name] = match[j]
    
    return df

In [67]:
df_old21 = find_ammo_qtty_description(df_old2)

df_old21.sample(1)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title_1,ammo_qtty_title_2,ammo_qtty_title_3,ammo_qtty_title_4,ammo_qtty_description_1,ammo_qtty_description_2,ammo_qtty_description_3,ammo_qtty_description_4,ammo_qtty_description_5,ammo_qtty_description_6,ammo_qtty_description_7,ammo_qtty_description_8,ammo_qtty_description_9,ammo_qtty_description_10,ammo_qtty_description_11,ammo_qtty_description_12
795,Cartouches FEDERAL - cal. 22LR - 36gr CPHP Hollow Point - Pot de 1375 cartouches,https://www.naturabuy.fr/Cartouches-FEDERAL-cal-22LR-36gr-CPHP-Hollow-Point-Pot-1375-cartouches-item-9472173.html,Federal,1,209.0,25.0,Marque : FederalEtat de l'objet : NeufType d'ogive : Cuivre Cartouches FEDERAL - cal. 22LR - 36gr CPHP Hollow Point - Pot de 1375 cartouches BYOB BULK PACKS Un supplément de frais de port sera appliqué pour tout envoi en Corse !,234.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# qtty of boxes search

# Regular expression pattern for finding numbers before/after French words that mean "box" or "boxes"
pattern3 = r'\b(\d+)\s*(?:(?:LOT|PAQUET)\s+DE\s+)?(?:(\d+)\s*(?:BOITE|BOITES))\b|\b(?:PAR\s+(\d+))\b'

# Function to find matching numbers and store in new columns
def find_box_numbers(df):
    # Find all matching numbers and store them in a list
    matches_name = df['product_name'].apply(lambda x: re.findall(pattern3, str(x)))
    matches_desc = df['product_description'].apply(lambda x: re.findall(pattern3, str(x)))
    matches = matches_name + matches_desc
    
    # Create new columns for each match and store the results
    for i, match in enumerate(matches):
        for j in range(len(match)):
            column_name = f'box_qty_{j+1}'
            df.at[i, column_name] = ''.join(match[j])
    
    return df

In [69]:
df_old22 = find_box_numbers(df_old21)

df_old22.sample(1)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title_1,ammo_qtty_title_2,ammo_qtty_title_3,ammo_qtty_title_4,ammo_qtty_description_1,ammo_qtty_description_2,ammo_qtty_description_3,ammo_qtty_description_4,ammo_qtty_description_5,ammo_qtty_description_6,ammo_qtty_description_7,ammo_qtty_description_8,ammo_qtty_description_9,ammo_qtty_description_10,ammo_qtty_description_11,ammo_qtty_description_12,box_qty_1,box_qty_2,box_qty_3,box_qty_4
287,Balles Federal Premium Match - Cal. 22 LR - Par 1 / 22LR / 40,https://www.naturabuy.fr/Balles-Federal-Premium-Match-Cal-22-LR-Par-1-item-9207325.html,Federal,1,31.25,9.9,"Marque : FederalEtat de l'objet : NeufType : MatchType d'ogive : Plomb Utilisez le meilleur pour la compétition. Ce sont les chargements d'usine les plus précis. Et c'est la technologie dont vous avez besoin pour ramener quelques médailles à la maison. /Caractéristiques: Calibre 22 LRBoîte50OgiveSolid100ms277Poids2.59Joules 100m100Joules 50m117Grains40 Quantité: Par 1, par 10 ou par 20 (voir options)",41.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
df.head()

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price,ammo_qtty_title_1,ammo_qtty_title_2,ammo_qtty_title_3,ammo_qtty_title_4,ammo_qtty_description_1,ammo_qtty_description_2,ammo_qtty_description_3,ammo_qtty_description_4,ammo_qtty_description_5,ammo_qtty_description_6,ammo_qtty_description_7,ammo_qtty_description_8,ammo_qtty_description_9,ammo_qtty_description_10,ammo_qtty_description_11,ammo_qtty_description_12,box_qty_1,box_qty_2,box_qty_3,box_qty_4
0,BOITE DE 325 CARTOUCHES 22LR FEDERAL GOLD PLATED HOLLOW POINT 36 GRAINS,https://www.naturabuy.fr/BOITE-DE-325-CARTOUCHES-22LR-FEDERAL-GOLD-PLATED-HOLLOW-POINT-36-GRAINS-item-10619103.html,Federal,1,46.9,12.0,Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Creuse boite de 325 cartouches de marque Federal 22lr 36 grains round nose hollow point cuivree excellente cartouche pour le tir dans les armes a verrou comme semi automatique marque : Fédéral référence : 725 Type d’ogive round nose cuivree a pointe creuse vitesse 384 mètres par secondes poids ogive : 36 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation,58.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Boîte ancienne 22 LR Remington 22 Target,https://www.naturabuy.fr/Boite-ancienne-22-LR-Remington-22-Target-item-10653966.html,Remington,0,11.0,4.4,Marque : RemingtonEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte en parfait état. Envoie par mondial relais 4€40,15.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"boite de 325 cartouches de 22Lr de marque Federal , Automatch , pour semi auto",https://www.naturabuy.fr/boite-325-cartouches-22Lr-marque-Federal-Automatch-semi-auto-item-10619099.html,Federal,1,44.9,12.0,"Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Plomb 1 boite de 325 cartouches de marque Federal 22lr ,40 grains round nose special semi auto Type d'ogive round nose en plomb vitesse 360 metres par secondes poids ogive : 40 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation",56.9,NaN,NaN,NaN,NaN,360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Boîte ancienne 22 LR Fiocchi expansive,https://www.naturabuy.fr/Boite-ancienne-22-LR-Fiocchi-expansive-item-10653962.html,Fiocchi,0,12.0,4.4,"Marque : FiocchiEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte ancienne, Fiocchi expansive, il manque 6 cartouches. Envoie par mondial relais 4,40 €",16.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Boîte ancienne 22 LR NCS 20 Match,https://www.naturabuy.fr/Boite-ancienne-22-LR-NCS-20-Match-item-10653952.html,NCS,0,20.0,4.4,"Marque : NCSEtat de l'objet : D'occasionType : MatchType d'ogive : Plomb Boîte ancienne frabriquée par la Nouvelle Cartoucherie de Survilliers (Cartoucherie Française). Boîte en parfait état. Envoie par mondial relay 4,40 €",24.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### old solution 3

In [ ]:
# read back raw data from csv
df = pd.read_csv('scraped_data_clean.csv')

In [ ]:
# V2 for bullet quantities

# col1 for qtty from title and description as a flat number
# col2 for num of boxes * 50
# if both are None we set default number of 50 in col3

def extract_bullet_qtty(text):
    # Check 1: total amount of bullets
    regex1 = r"\b(\d+)\s*boîte(?:s)?\s*de\s*(50|525)\s*(?:cartouches|balles|munitions)\b|\bMunition \/ boite\s*:\s*(\d+)\s*x\s*(50|525)\b|\bQuantité\s*:\s*(50|525)\b"
    match1 = re.search(regex1, text, re.IGNORECASE)
    if match1:
        qtty = int(match1.group(1)) if match1.group(1) is not None else int(match1.group(2))
        return qtty
    # Check 2: amount of boxes
        regex2 = r"\bLOT DE (\d+) BOITES\b"
    match2 = re.search(regex2, text, re.IGNORECASE)
    if match2:
        qtty = int(match2.group(1)) * 50
        return qtty
    else:
        return 0

# create new columns for ammo quantity
df["bullet_qtty_v1"] = df.apply(lambda x: extract_bullet_qtty(x["product_name"]) or extract_bullet_qtty(x["product_description"]) or 0)
df["bullet_qtty_v2"] = df.apply(lambda x: extract_bullet_qtty(x["product_name"]) * 50 if re.search(r"\b(\d+)\s*boites\b", x["product_name"]) else extract_bullet_qtty(x["product_description"]) * 50 if re.search(r"\b(\d+)\s*boites\b", x["product_description"]) else 0, axis=1)

# set default value of 50 for products with missing values
df["bullet_qtty_default"] = df.apply(lambda x: 50 if x["bullet_qtty_v1"] == 0 and x["bullet_qtty_v2"] == 0 else None, axis=1)

df.head(5)

In [ ]:
# Function to extract numbers divisible by 50 from a given text
def extract_divisible_by_50(text):
    numbers = re.findall(r'\b(?:[5-9][0]|50)\b', text)
    return [int(num) for num in numbers]

# Apply the function to "product_name" and "product_description" columns and store the result in "ammo_qtty" column
df['ammo_qtty'] = df['product_name'].apply(extract_divisible_by_50) + df['product_description'].apply(extract_divisible_by_50)

# Filter and keep only the numbers divisible by 50 without remainders
df['ammo_qtty'] = df['ammo_qtty'].apply(lambda x: [num for num in x if num % 50 == 0])

In [ ]:
df.ammo_qtty[:10]

In [ ]:
df.columns